In [ ]:
# =========================================================
# Lysosome detection + neuron segmentation (CZI or TIFF)
# with DT-refined continuous radii (2D per-slice)
# + per-cell Z-axis rotation videos (MIP overlays)
# =========================================================

import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball, binary_erosion
)
from scipy.ndimage import distance_transform_edt as edt
from scipy.ndimage import rotate as ndrotate
from skimage.measure import label, regionprops
from skimage.segmentation import watershed, find_boundaries
from scipy.ndimage import binary_fill_holes

import napari

# ---------------------------
# Helper: refine radii by DT
# ---------------------------
def refine_radii_via_dt(img3d, blobs, win_px=25, bin_method="sauvola"):
    """
    Refine per-blob radius using a local 2D binary mask + distance transform on the Z slice.
    - img3d: float32 (Z, Y, X), same you used for LoG (e.g., smoothed)
    - blobs: ndarray [N, 4] with columns [z, y, x, r_px] (r is initial estimate; overwritten)
    - win_px: half-window around (y,x) to build local mask
    - bin_method: 'sauvola' | 'local' | 'otsu'
    Returns: blobs_refined with same centers [z,y,x] and r_px from DT (continuous).
    """
    from skimage.filters import threshold_sauvola, threshold_local, threshold_otsu
    from skimage.morphology import remove_small_objects, disk, binary_opening
    from scipy.ndimage import distance_transform_edt as _edt
    from skimage.measure import label as _label

    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z = int(round(float(zc)))
        y = int(round(float(yc)))
        x = int(round(float(xc)))

        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        if (y2 <= y1) or (x2 <= x1):
            continue

        patch = img3d[z, y1:y2, x1:x2]

        # Adaptive/local binarization
        if bin_method == "sauvola":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_sauvola(patch, window_size=ws, k=0.2)
            bw = patch > thr
        elif bin_method == "local":
            ws = max(21, 2*(win_px//2)+1)
            thr = threshold_local(patch, block_size=ws, offset=-0.2*np.std(patch))
            bw = patch > thr
        else:  # 'otsu'
            try:
                thr = threshold_otsu(patch)
                bw = patch > thr
            except ValueError:
                continue

        # light clean-up
        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        # Keep only the component that contains (or is nearest to) the center
        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            found = False
            for dy in (-1, 0, 1):
                for dx in (-1, 0, 1):
                    yyy, xxx = yy + dy, xx + dx
                    if 0 <= yyy < bw.shape[0] and 0 <= xxx < bw.shape[1] and bw[yyy, xxx]:
                        yy, xx = yyy, xxx
                        found = True
                        break
                if found:
                    break
            if not found:
                continue

        lab = _label(bw, connectivity=2)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)

        # DT-based radius (continuous, in pixels on this slice)
        dt = _edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px <= 0:
            continue

        out[i, 3] = r_px  # overwrite radius in pixels (continuous)

    return out
# ==========================================
# CONFIG: set a file path (.czi or .tif/.tiff)
# ==========================================
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.tif"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.tif"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/ISP MAPATZ 2025/TMEM-HA 6h CB airy_3_170722.tif"

# ==========================================
# Generic loader for CZI and TIFF (OME/ImageJ)
# Returns: ch1, ch2, (vx_um, vy_um, vz_um), meta
# ==========================================
def _try_float(x, default=None):
    try:
        return float(x)
    except Exception:
        return default

def _parse_ome_xml(xml_text: str):
    """Parse OME-XML PhysicalSizeX/Y/Z → µm (graceful fallback)."""
    if not xml_text:
        return None
    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"(?:\s+PhysicalSize{attr}Unit="([^"]+)")?', xml_text)
        if not m:
            return None, None
        return m.group(1), (m.group(2) or "")

    x_val, x_unit = grab('X')
    y_val, y_unit = grab('Y')
    z_val, z_unit = grab('Z')

    def to_um(val_str, unit):
        v = _try_float(val_str)
        if v is None:
            return None
        u = unit.lower()
        if u in ("µm", "um", "micrometer", "micrometre", "microns", "micron"):
            return v
        if u in ("m", "meter", "metre"):
            return v * 1e6
        # If unit missing: treat tiny as meters, else µm
        return v * 1e6 if v < 1e-3 else v

    vx = to_um(x_val, x_unit) if x_val is not None else None
    vy = to_um(y_val, y_unit) if y_val is not None else None
    vz = to_um(z_val, z_unit) if z_val is not None else None
    return vx, vy, vz

def _voxel_from_tiff_tags(tf: tiff.TiffFile):
    """
    Try ImageJ-style X/Y resolution (inch/cm → µm/px) and Z spacing from ImageDescription.
    Returns (vx_um, vy_um, vz_um) with None if not found.
    """
    vx = vy = vz = None
    try:
        page0 = tf.pages[0]
        xres = getattr(page0, "tags", {}).get("XResolution", None)
        yres = getattr(page0, "tags", {}).get("YResolution", None)
        resunit = getattr(page0, "tags", {}).get("ResolutionUnit", None)

        def res_to_um(res_tag, unit_tag):
            if res_tag is None:
                return None
            val = res_tag.value
            if isinstance(val, tuple) and len(val) == 2:
                num, den = val
            else:
                try:
                    num, den = val.numerator, val.denominator
                except Exception:
                    return None
            if den == 0:
                return None
            ppu = num / den  # pixels per unit
            if ppu <= 0:
                return None
            unit = (unit_tag.value if unit_tag else 2)  # 2=inches, 3=cm
            if unit == 2:
                um_per_unit = 25400.0
            elif unit == 3:
                um_per_unit = 10000.0
            else:
                return None
            return um_per_unit / ppu  # µm per pixel

        vx = res_to_um(xres, resunit)
        vy = res_to_um(yres, resunit)

        # Try ImageDescription for Z spacing (often in µm)
        desc = page0.tags.get("ImageDescription", None)
        if desc is not None:
            txt = str(desc.value)
            m = re.search(r'(spacing|SliceSpacing)[=:]\s*([0-9.+-eE]+)', txt)
            if m:
                vz = _try_float(m.group(2))
    except Exception:
        pass
    return vx, vy, vz

def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    vx_um = vy_um = vz_um = None

    if ext == ".czi":
        # -------- CZI --------
        with czifile.CziFile(file_path) as czi:
            img = czi.asarray()
            meta_xml = czi.metadata()
        img = np.squeeze(img)

        # Infer channels
        if img.ndim < 3:
            raise RuntimeError(f"CZI has unexpected ndim={img.ndim}")
        if img.shape[0] == 2:        # (C, Z, Y, X)
            ch1, ch2 = img[0], img[1]
        elif img.shape[1] == 2:      # (Z, C, Y, X)
            ch1, ch2 = img[:, 0], img[:, 1]
        else:
            raise RuntimeError(f"Can't auto-detect 2 channels in CZI shape {img.shape}")

        # Voxel size from CZI XML
        vx_um = vy_um = vz_um = 1.0
        try:
            r = ET.fromstring(meta_xml)
            def _get_um(axis: str) -> float:
                v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
                u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
                val = float(v.text) if v is not None else 1.0
                unit = (u.text or "").lower() if u is not None else ""
                if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
                    val *= 1e6
                elif unit in ("µm", "um", "micrometer", "micrometre"):
                    pass
                else:
                    if val < 1e-3:
                        val *= 1e6
                return val
            vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
        except Exception:
            pass
        return ch1, ch2, (vx_um or 1.0, vy_um or 1.0, vz_um or 1.0), {"type": "czi"}

    elif ext in (".tif", ".tiff"):
        # -------- TIFF / OME-TIFF --------
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            ome_xml = None
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                pass

            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml) or (None, None, None)
            if vx_um is None or vy_um is None or vz_um is None:
                tx, ty, tz = _voxel_from_tiff_tags(tf)
                vx_um = vx_um if vx_um is not None else tx
                vy_um = vy_um if vy_um is not None else ty
                vz_um = vz_um if vz_um is not None else tz

        img = np.squeeze(arr)

        # Layouts we support:
        if img.ndim == 4:
            if img.shape[0] == 2:        # (C, Z, Y, X)
                ch1, ch2 = img[0], img[1]
            elif img.shape[1] == 2:      # (Z, C, Y, X)
                ch1, ch2 = img[:, 0], img[:, 1]
            elif img.shape[-1] == 2:     # (Z, Y, X, C)
                ch1, ch2 = img[..., 0], img[..., 1]
            else:
                raise RuntimeError(f"Cannot infer 2 channels from TIFF shape {img.shape}")
        elif img.ndim == 3:
            if img.shape[-1] == 2:       # (Y, X, C) → promote Z=1
                ch1 = img[..., 0][None, ...]
                ch2 = img[..., 1][None, ...]
            else:
                raise RuntimeError("TIFF is single-channel; expected 2 channels.")
        else:
            raise RuntimeError(f"Unsupported TIFF ndim: {img.ndim}")

        return ch1, ch2, (vx_um or 1.0, vy_um or 1.0, vz_um or 1.0), {"type": "tiff", "ome": bool(ome_xml)}

    else:
        raise ValueError(f"Unsupported file extension: {ext}")

# ==========================================
# 1) Load image (CZI or TIFF) + voxel size (µm)
# ==========================================
img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# Per-voxel metrics
voxel_um3    = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)              # linear scale preserving volume
print(f"[{meta.get('type','?').upper()}] Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g} | equiv linear µm={lin_equiv_um:.4g}")
print("Ch1 shape:", image.shape, "| Ch2 shape:", image_2.shape)

# ==========================================
# 2) CH1: lysosome centers (LoG) + DT-refined radii (continuous)
# ==========================================
image_smooth = gaussian(image, sigma=1.5)

blobs = blob_log(
    #image,
    image_smooth,
    min_sigma=0.8,
    max_sigma=3,
    num_sigma=4,
    threshold=0.003,#0.006
    overlap=1#1
    #log_scale=False#False
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)
print(f"Detected {len(blobs)} lysosomes (LoG centers).")

image_smooth = image_smooth.astype(np.float32, copy=False)
blobs = refine_radii_via_dt(image_smooth, blobs, win_px=25, bin_method="sauvola")

# Convert per-lysosome metrics to physical units (µm)
if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um

    # Continuous radius from DT (in XY pixels) → microns
    #xy_um = 0.5 * (vx_um + vy_um)  # if isotropic XY, you can use vx_um
    #xy_um = 0.5 * vx_um 
    radius_um   = (blobs[:, 3] * vx_um)*0.1
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

# Per-blob DF (µm only)
blob_ids = np.arange(1, len(blobs) + 1)
df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only, radii refined by DT)")

# ==========================================
# 3) Viewer base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE)
# ==========================================
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0
ch2 = gaussian(vol, sigma=1.5, preserve_range=True)

# Local threshold per z
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=301, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t
neuron_mask = remove_small_objects(neuron_mask, min_size=20000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(10))
neuron_mask = binary_erosion(neuron_mask, ball(6))

# Soma via distance + cleanup
dist = edt(neuron_mask)
cell_min_radius_vox = 1
cell_mask = (dist >= cell_min_radius_vox)
cell_mask &= neuron_mask
cell_mask = binary_opening(cell_mask, ball(6))
cell_mask = binary_closing(cell_mask, ball(10))
cell_mask = binary_fill_holes(cell_mask)

# Label and territories
body_lab = label(cell_mask, connectivity=3)
n_cells = int(body_lab.max())
print(f"Detected {n_cells} cells (soma).")

if n_cells > 0:
    dist_inside = edt(neuron_mask)
    cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels:", int(cell_mask.sum()))

# ==========================================
# ADD-ON: optional filter out oversized cell bodies
# ==========================================
MAX_BODY_VOXELS = 65000  # set >0 to enable filtering
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0 and MAX_BODY_VOXELS > 0:
        counts = np.bincount(cell_seg.ravel().astype(np.int64))
        drop_labels = np.where(counts < MAX_BODY_VOXELS)[0]
        drop_labels = drop_labels[drop_labels != 0]

        if drop_labels.size > 0:
            print(f"Filtered {len(drop_labels)} oversized cells. IDs: {drop_labels.tolist()}")
            to_remove = np.isin(cell_seg, drop_labels)
            cell_seg[to_remove] = 0
            if 'cell_mask' in globals() and isinstance(cell_mask, np.ndarray):
                cell_mask[to_remove] = False

            pd.DataFrame({
                "filtered_cell_id_ch2": drop_labels,
                "voxel_count": counts[drop_labels]
            }).to_csv("filtered_cells_gt_threshold.csv", index=False)
        else:
            print(f"No cells exceeded {MAX_BODY_VOXELS} voxels; nothing filtered.")
except Exception as e:
    print("Cell size filtering failed:", e)

# ==========================================
# 5) Map lysosomes to (cell/outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue
        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Include zero-count cells
try:
    if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
        all_cells = pd.DataFrame({
            "cell_id_ch2": np.arange(1, int(cell_seg.max()) + 1, dtype=int)
        })

        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts_nonzero = (
                df[df["location_ch2"] == "cell"]
                .groupby("cell_id_ch2")
                .size()
                .reset_index(name="count")
            )
        else:
            lys_counts_nonzero = pd.DataFrame(columns=["cell_id_ch2", "count"])

        lys_counts_all = (
            all_cells.merge(lys_counts_nonzero, on="cell_id_ch2", how="left")
                     .fillna({"count": 0})
        )
        lys_counts_all["count"] = lys_counts_all["count"].astype(int)

        lys_counts_all.to_csv("lysosome_counts_by_cell.csv", index=False)
        print("Updated: lysosome_counts_by_cell.csv now includes cells with 0 lysosomes.")
except Exception as e:
    print("Could not expand lysosome_counts_by_cell with zero-count cells:", e)

# ==========================================
# 5b) Per-cell (Ch2) volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 6) Visualization (CELL vs OUTSIDE only)
# ==========================================
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'

    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass
"""
# Lysosome points overlay
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if len(blobs) > 0 and loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES"
except Exception:
    pass
"""
# Lysosome points overlay (show ONLY lysosomes inside cells)
if len(blobs) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    in_cell_mask = (df["location_ch2"].to_numpy() == "cell")
    if in_cell_mask.any():
        blobs_cell = blobs[in_cell_mask, :3]            # z,y,x
        radii_cell = blobs[in_cell_mask, 3] if blobs.shape[1] > 3 else np.ones(np.count_nonzero(in_cell_mask))
        pts = viewer.add_points(
            blobs_cell,
            size=np.clip(radii_cell * 2, 2, None),
            name='Lysosomes (cell only)'
        )
        try:
            pts.face_color = [0.0, 1.0, 1.0, 1.0]       # cyan
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df.loc[in_cell_mask, 'id'].to_numpy(),
                'cell':   df.loc[in_cell_mask, 'cell_id_ch2'].to_numpy(),
                'diameter_um': df.loc[in_cell_mask, 'diameter_um'].to_numpy(),
                'volume_um3':  df.loc[in_cell_mask, 'volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
        except Exception:
            pass

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================

img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)
    """
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)
    """
    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        # iterate only over blobs that are inside the cell on this slice
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                # draw only in-cell lysosomes (cyan/yellow are both OK; keeping yellow for visibility)
                cv2.circle(overlay, (x, y), r, (255, 255, 0), 2)
        # NOTE: no drawing at all for outside points
    
    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")


# ==========================================
# 8) Per-cell Z-rotation videos (MIP overlays)
# ==========================================
"""
def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def save_cell_spin_videos(ch2_raw, cell_labels, out_dir="cell_spin_videos",
                          angles=tuple(range(0, 360, 10)), pad=8, fps=12):
    
    #For each cell id in 'cell_labels', extract a padded subvolume,
    #rotate around Z (axes=(1,2)), compute Z-MIP per angle, and save MP4.
    
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    os.makedirs(out_dir, exist_ok=True)

    # regionprops gives bounding boxes in (min_z, min_y, min_x, max_z, max_y, max_x)
    props = regionprops(cell_labels)
    exported = 0

    for p in props:
        cid = p.label
        if cid == 0:
            continue
        minz, miny, minx, maxz, maxy, maxx = p.bbox

        # add padding within bounds
        Z, Y, X = cell_labels.shape
        minz = max(0, minz - 0)
        maxz = min(Z, maxz + 0)
        miny = max(0, miny - pad)
        maxy = min(Y, maxy + pad)
        minx = max(0, minx - pad)
        maxx = min(X, maxx + pad)

        sub_lab = (cell_labels[minz:maxz, miny:maxy, minx:maxx] == cid)
        if sub_lab.sum() == 0:
            continue

        sub_vol = ch2_raw[minz:maxz, miny:maxy, minx:maxx]

        frames = []
        for ang in angles:
            # rotate XY plane (Y,X) -> axes=(1,2)
            rot_vol = ndrotate(sub_vol, angle=ang, axes=(1,2), reshape=False, order=1, mode='nearest')
            rot_msk = ndrotate(sub_lab.astype(np.uint8), angle=ang, axes=(1,2),
                               reshape=False, order=0, mode='nearest') > 0

            # show only the cell voxels in MIP (masking background)
            rot_vol_masked = np.where(rot_msk, rot_vol, 0.0)
            mip = rot_vol_masked.max(axis=0)   # (Y,X)
            msk_mip = rot_msk.max(axis=0)      # (Y,X) bool

            base_u8 = _to_uint8(mip)
            base_bgr = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)

            # green overlay for cell footprint
            if msk_mip.any():
                cell_u8 = (msk_mip.astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell_u8)
                frame = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                frame = base_bgr

            # optional: thin outline for readability
            contours, _ = cv2.findContours((msk_mip.astype(np.uint8)*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(frame, contours, -1, (0, 255, 0), 1, lineType=cv2.LINE_AA)

            # put ID + angle text
            cv2.putText(frame, f"Cell {cid}  angle {ang:03d}°", (8, 18),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (240, 240, 240), 1, cv2.LINE_AA)

            frames.append(frame)

        out_mp4 = os.path.join(out_dir, f"cell_{cid:03d}_spinZ.mp4")
        try:
            imageio.mimsave(out_mp4, [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                            fps=fps, format='FFMPEG')
            print(f"Saved: {out_mp4}")
        except TypeError:
            out_gif = os.path.join(out_dir, f"cell_{cid:03d}_spinZ.gif")
            imageio.mimsave(out_gif, [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=fps)
            print(f"Saved (GIF): {out_gif}")

        exported += 1

    if exported == 0:
        print("No per-cell videos exported (no cells found).")
    else:
        print(f"Exported {exported} per-cell spin videos to: {out_dir}")

# call it (uses raw ch2 intensity and the watershed labels)
save_cell_spin_videos(image_2.astype(np.float32), cell_seg, out_dir="cell_spin_videos",
                      angles=tuple(range(0, 360, 10)), pad=8, fps=12)
"""
########################################################################
# ==========================================
# 8) Per-cell rotation videos (X, Y, Z axes) – only for cells with lysosomes
# ==========================================

from scipy.ndimage import rotate as ndrotate

def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    #Rotate 3D points around chosen axis (X, Y, or Z).
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":     # rotate in ZX plane
        z = p[:, 0]*c + p[:, 2]*s
        y = p[:, 1]
        x = -p[:, 0]*s + p[:, 2]*c
    elif axis.upper() == "X":   # rotate in YZ plane
        z = p[:, 0]
        y = p[:, 1]*c - p[:, 2]*s
        x = p[:, 1]*s + p[:, 2]*c
    else:                       # Z axis (rotate in XY plane)
        z = p[:, 0]
        y = p[:, 1]*c + p[:, 2]*s
        x = -p[:, 1]*s + p[:, 2]*c
    return np.stack([z, y, x], axis=1) + center_zyx

def save_cell_spin_videos_allaxes(cell_labels, blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):
    
    #Create per-cell rotation videos around X, Y, and Z axes.
    #Only exports cells that contain lysosomes.
    #Shows:
    #  - Green cell footprint
    #  - Cyan lysosome points inside cell
    #  - Cell ID text
    
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, COL_GREEN, TXT_COL = (255, 255, 0), (0, 255, 0), (240, 240, 240)
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    Ztot, Ytot, Xtot = cell_labels.shape
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)
        sub_lab = cell_labels[minz:maxz, miny:maxy, minx:maxx]
        if (sub_lab == cid).sum() == 0:
            continue
        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        # lysosomes in this cell
        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        # iterate over each axis type
        for axis in axes_all:
            frames = []
            for ang in angles:
                # rotate volume around chosen axis
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                else:  # Z
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')

                mask_rot = (rot_lab > 0)
                footprint = mask_rot.max(axis=0).astype(np.uint8)
                frame = np.zeros((subY, subX, 3), dtype=np.uint8)

                # green overlay
                if footprint.any():
                    green = (footprint * 255)
                    overlay = frame.copy()
                    overlay[..., 1] = np.maximum(overlay[..., 1], green)
                    frame = cv2.addWeighted(frame, 1.0, overlay, 0.35, 0.0)
                    contours, _ = cv2.findContours((footprint*255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    if contours:
                        cv2.drawContours(frame, contours, -1, COL_GREEN, 1, lineType=cv2.LINE_AA)

                #cv2.putText(frame, f"Cell {cid}  {axis}-rot {ang:03d}°", (8, 18),
                #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, TXT_COL, 1, cv2.LINE_AA)

                # rotate and draw lysosome points
                pts_rot = _rotate_points(pts_local, center_zyx, ang, axis=axis)
                ys = np.clip(np.round(pts_rot[:, 1]).astype(int), 0, subY-1)
                xs = np.clip(np.round(pts_rot[:, 2]).astype(int), 0, subX-1)
                for k in range(len(xs)):
                    cv2.circle(frame, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                frames.append(frame)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps)
                print(f"Saved (GIF): {out_gif}")
            exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")


# ---- call it ----
save_cell_spin_videos_allaxes(cell_seg, blobs, df,
                              out_dir="cell_spin_videos",
                              angles=tuple(range(0, 360, 10)),
                              pad=8, fps=12)
"""
from scipy.ndimage import rotate as ndrotate

def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    #Rotate 3D points around chosen axis (X, Y, or Z).
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":     # rotate in ZX plane
        z =  c*p[:, 0] + s*p[:, 2]
        y =  p[:, 1]
        x = -s*p[:, 0] + c*p[:, 2]
    elif axis.upper() == "X":   # rotate in YZ plane
        z =  p[:, 0]
        y =  c*p[:, 1] - s*p[:, 2]
        x =  s*p[:, 1] + c*p[:, 2]
    else:                       # Z axis (rotate in XY plane)
        z =  p[:, 0]
        y =  c*p[:, 1] + s*p[:, 2]
        x = -s*p[:, 1] + c*p[:, 2]
    return np.stack([z, y, x], axis=1) + center_zyx

def _edge_enhance_u8(gray_u8, alpha=0.35):
    #Slightly boost edges to emphasize surface changes (Sobel).
    g = gray_u8.astype(np.float32)
    gx = cv2.Sobel(g, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(g, cv2.CV_32F, 0, 1, ksize=3)
    mag = np.sqrt(gx*gx + gy*gy)
    if mag.max() > 0:
        mag = mag * (255.0 / mag.max())
    out = np.clip(g + alpha * mag, 0, 255).astype(np.uint8)
    return out

def save_cell_spin_videos_allaxes(ch2_raw,         # <-- NEW: pass the raw Ch2 volume (Z,Y,X)
                                  cell_labels,
                                  blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):
    
    #Create per-cell rotation videos around X, Y, and Z axes.
    #Only exports cells that contain lysosomes.
    #Shows per-angle *textured* cell surface using masked raw-intensity MIP,
    #plus cell outline and lysosome points.
    
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, TXT_COL = (255, 255, 0), (240, 240, 240)  # cyan-ish points, light text
    Ztot, Ytot, Xtot = cell_labels.shape
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        # pad in XY (not Z) and clamp
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)

        sub_lab = (cell_labels[minz:maxz, miny:maxy, minx:maxx] == cid)
        if sub_lab.sum() == 0:
            continue
        sub_vol = ch2_raw[minz:maxz, miny:maxy, minx:maxx].astype(np.float32)

        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        # lysosomes in this cell -> local coordinates
        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        def make_spin(axis):
            frames = []
            for ang in angles:
                # rotate label and raw subvolume together
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                    rot_vol = ndrotate(sub_vol, angle=ang, axes=(0,2), reshape=False, order=1, mode='nearest')
                    # MIP over Y → (Z,X) image
                    rot_mask = rot_lab > 0
                    mip = np.where(rot_mask, rot_vol, 0).max(axis=1)   # (Z,X)
                    mask_mip = rot_mask.max(axis=1).astype(np.uint8)   # (Z,X)
                    # arrange canvas as (rows=Z, cols=X)
                    base_u8 = _to_uint8(mip)
                    canvas  = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)
                    # rotate lysosome points and project (z,x)
                    pts_rot = _rotate_points(pts_local, center_zyx, ang, axis="Y")
                    pts_draw = np.stack([pts_rot[:, 0], pts_rot[:, 2]], axis=1)  # (z,x)
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                    rot_vol = ndrotate(sub_vol, angle=ang, axes=(1,0), reshape=False, order=1, mode='nearest')
                    # MIP over X → (Z,Y) image; display as (Y,Z) for conventional (rows, cols)
                    rot_mask = rot_lab > 0
                    mip = np.where(rot_mask, rot_vol, 0).max(axis=2)   # (Z,Y)
                    mask_mip = rot_mask.max(axis=2).astype(np.uint8)   # (Z,Y)
                    base_u8 = _to_uint8(mip)
                    base_u8 = base_u8.T              # (Y,Z)
                    mask_mip = mask_mip.T            # (Y,Z)
                    canvas  = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)
                    pts_rot = _rotate_points(pts_local, center_zyx, ang, axis="X")
                    pts_draw = np.stack([pts_rot[:, 1], pts_rot[:, 0]], axis=1)  # (y,z) because of transpose
                else:  # "Z"
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')
                    rot_vol = ndrotate(sub_vol, angle=ang, axes=(1,2), reshape=False, order=1, mode='nearest')
                    # MIP over Z → (Y,X) image
                    rot_mask = rot_lab > 0
                    mip = np.where(rot_mask, rot_vol, 0).max(axis=0)   # (Y,X)
                    mask_mip = rot_mask.max(axis=0).astype(np.uint8)   # (Y,X)
                    base_u8 = _to_uint8(mip)
                    canvas  = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)
                    pts_rot = _rotate_points(pts_local, center_zyx, ang, axis="Z")
                    pts_draw = np.stack([pts_rot[:, 1], pts_rot[:, 2]], axis=1)  # (y,x)

                # subtle edge enhancement to highlight surface relief
                canvas[..., 0] = _edge_enhance_u8(canvas[..., 0])
                canvas[..., 1] = _edge_enhance_u8(canvas[..., 1])
                canvas[..., 2] = _edge_enhance_u8(canvas[..., 2])

                # Outline of the projected cell mask (thin)
                m_u8 = (mask_mip * 255).astype(np.uint8)
                contours, _ = cv2.findContours(m_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    cv2.drawContours(canvas, contours, -1, (0, 255, 0), 1, lineType=cv2.LINE_AA)

                # Lysosome points (always drawn; cyan-ish). Clip to canvas.
                h, w = canvas.shape[:2]
                ys = np.clip(np.round(pts_draw[:, 0]).astype(int), 0, h-1)
                xs = np.clip(np.round(pts_draw[:, 1]).astype(int), 0, w-1)
                for k in range(len(xs)):
                    cv2.circle(canvas, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                # Label
                cv2.putText(canvas, f"Cell {cid}  {axis}-rot {ang:03d}°",
                            (8, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, TXT_COL, 1, cv2.LINE_AA)

                frames.append(canvas)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps)
                print(f"Saved (GIF): {out_gif}")

        # Make the three spins
        make_spin("Z")
        make_spin("Y")
        make_spin("X")
        exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")
###############################################################333
from scipy.ndimage import rotate as ndrotate

def _to_uint8(vol):
    v = vol.astype(np.float32)
    vmin, vmax = float(v.min()), float(v.max())
    if vmax <= vmin:
        return np.zeros_like(v, dtype=np.uint8)
    return (255.0 * (v - vmin) / (vmax - vmin)).astype(np.uint8)

def _rotate_points(pts_zyx, center_zyx, angle_deg, axis="Y"):
    #Rotate 3D points around chosen axis (X, Y, or Z).
    if pts_zyx.size == 0:
        return pts_zyx
    theta = np.deg2rad(angle_deg)
    c, s = np.cos(theta), np.sin(theta)
    p = pts_zyx - center_zyx
    if axis.upper() == "Y":     # rotate in ZX plane
        z =  c*p[:, 0] + s*p[:, 2]
        y =  p[:, 1]
        x = -s*p[:, 0] + c*p[:, 2]
    elif axis.upper() == "X":   # rotate in YZ plane
        z =  p[:, 0]
        y =  c*p[:, 1] - s*p[:, 2]
        x =  s*p[:, 1] + c*p[:, 2]
    else:                       # Z axis (rotate in XY plane)
        z =  p[:, 0]
        y =  c*p[:, 1] + s*p[:, 2]
        x = -s*p[:, 1] + c*p[:, 2]
    return np.stack([z, y, x], axis=1) + center_zyx

def _edge_enhance_u8(gray_u8, alpha=0.35):
    #Slightly boost edges to emphasize surface changes (Sobel).
    g = gray_u8.astype(np.float32)
    gx = cv2.Sobel(g, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(g, cv2.CV_32F, 0, 1, ksize=3)
    mag = np.sqrt(gx*gx + gy*gy)
    if mag.max() > 0:
        mag = mag * (255.0 / mag.max())
    out = np.clip(g + alpha * mag, 0, 255).astype(np.uint8)
    return out

def save_cell_spin_videos_allaxes(ch2_raw,         # <-- NEW: pass the raw Ch2 volume (Z,Y,X)
                                  cell_labels,
                                  blobs, df,
                                  out_dir="cell_spin_videos",
                                  angles=tuple(range(0, 360, 10)),
                                  pad=8, fps=12,
                                  draw_point_radius_px=3):
    
    #Create per-cell rotation videos around X, Y, and Z axes.
    #Only exports cells that contain lysosomes.
    #Shows per-angle *textured* cell surface using masked raw-intensity MIP,
    #plus cell outline and lysosome points.
    
    if not isinstance(cell_labels, np.ndarray) or cell_labels.max() == 0:
        print("No cells to export.")
        return
    if not isinstance(df, pd.DataFrame) or "cell_id_ch2" not in df.columns:
        print("Missing per-blob metadata; cannot filter by lysosome presence.")
        return

    cell_ids_with_lys = sorted(df.loc[
        (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0),
        "cell_id_ch2"
    ].unique().astype(int))
    if len(cell_ids_with_lys) == 0:
        print("No cells contain lysosomes; no videos exported.")
        return

    os.makedirs(out_dir, exist_ok=True)
    COL_CYAN, TXT_COL = (255, 255, 0), (240, 240, 240)  # cyan-ish points, light text
    Ztot, Ytot, Xtot = cell_labels.shape
    props = [p for p in regionprops(cell_labels) if p.label in cell_ids_with_lys]
    axes_all = ["X", "Y", "Z"]
    exported = 0

    for p in props:
        cid = p.label
        minz, miny, minx, maxz, maxy, maxx = p.bbox
        # pad in XY (not Z) and clamp
        miny, maxy = max(0, miny - pad), min(Ytot, maxy + pad)
        minx, maxx = max(0, minx - pad), min(Xtot, maxx + pad)

        sub_lab = (cell_labels[minz:maxz, miny:maxy, minx:maxx] == cid)
        if sub_lab.sum() == 0:
            continue
        sub_vol = ch2_raw[minz:maxz, miny:maxy, minx:maxx].astype(np.float32)

        subZ, subY, subX = sub_lab.shape
        center_zyx = np.array([(subZ-1)/2, (subY-1)/2, (subX-1)/2], dtype=np.float32)

        # lysosomes in this cell -> local coordinates
        sub_blobs = df[(df["cell_id_ch2"] == cid) & (df["location_ch2"] == "cell")]
        if sub_blobs.empty:
            continue
        sel = sub_blobs.index.to_numpy()
        pts_local = np.stack([
            blobs[sel, 0]-minz,
            blobs[sel, 1]-miny,
            blobs[sel, 2]-minx
        ], axis=1).astype(np.float32)

        def make_spin(axis):
            frames = []
            for ang in angles:
                # rotate label and raw subvolume together
                if axis == "Y":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(0,2), reshape=False, order=0, mode='nearest')
                    rot_vol = ndrotate(sub_vol, angle=ang, axes=(0,2), reshape=False, order=1, mode='nearest')
                    # MIP over Y → (Z,X) image
                    rot_mask = rot_lab > 0
                    mip = np.where(rot_mask, rot_vol, 0).max(axis=1)   # (Z,X)
                    mask_mip = rot_mask.max(axis=1).astype(np.uint8)   # (Z,X)
                    # arrange canvas as (rows=Z, cols=X)
                    base_u8 = _to_uint8(mip)
                    canvas  = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)
                    # rotate lysosome points and project (z,x)
                    pts_rot = _rotate_points(pts_local, center_zyx, ang, axis="Y")
                    pts_draw = np.stack([pts_rot[:, 0], pts_rot[:, 2]], axis=1)  # (z,x)
                elif axis == "X":
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,0), reshape=False, order=0, mode='nearest')
                    rot_vol = ndrotate(sub_vol, angle=ang, axes=(1,0), reshape=False, order=1, mode='nearest')
                    # MIP over X → (Z,Y) image; display as (Y,Z) for conventional (rows, cols)
                    rot_mask = rot_lab > 0
                    mip = np.where(rot_mask, rot_vol, 0).max(axis=2)   # (Z,Y)
                    mask_mip = rot_mask.max(axis=2).astype(np.uint8)   # (Z,Y)
                    base_u8 = _to_uint8(mip)
                    base_u8 = base_u8.T              # (Y,Z)
                    mask_mip = mask_mip.T            # (Y,Z)
                    canvas  = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)
                    pts_rot = _rotate_points(pts_local, center_zyx, ang, axis="X")
                    pts_draw = np.stack([pts_rot[:, 1], pts_rot[:, 0]], axis=1)  # (y,z) because of transpose
                else:  # "Z"
                    rot_lab = ndrotate(sub_lab, angle=ang, axes=(1,2), reshape=False, order=0, mode='nearest')
                    rot_vol = ndrotate(sub_vol, angle=ang, axes=(1,2), reshape=False, order=1, mode='nearest')
                    # MIP over Z → (Y,X) image
                    rot_mask = rot_lab > 0
                    mip = np.where(rot_mask, rot_vol, 0).max(axis=0)   # (Y,X)
                    mask_mip = rot_mask.max(axis=0).astype(np.uint8)   # (Y,X)
                    base_u8 = _to_uint8(mip)
                    canvas  = cv2.cvtColor(base_u8, cv2.COLOR_GRAY2BGR)
                    pts_rot = _rotate_points(pts_local, center_zyx, ang, axis="Z")
                    pts_draw = np.stack([pts_rot[:, 1], pts_rot[:, 2]], axis=1)  # (y,x)

                # subtle edge enhancement to highlight surface relief
                canvas[..., 0] = _edge_enhance_u8(canvas[..., 0])
                canvas[..., 1] = _edge_enhance_u8(canvas[..., 1])
                canvas[..., 2] = _edge_enhance_u8(canvas[..., 2])

                # Outline of the projected cell mask (thin)
                m_u8 = (mask_mip * 255).astype(np.uint8)
                contours, _ = cv2.findContours(m_u8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    cv2.drawContours(canvas, contours, -1, (0, 255, 0), 1, lineType=cv2.LINE_AA)

                # Lysosome points (always drawn; cyan-ish). Clip to canvas.
                h, w = canvas.shape[:2]
                ys = np.clip(np.round(pts_draw[:, 0]).astype(int), 0, h-1)
                xs = np.clip(np.round(pts_draw[:, 1]).astype(int), 0, w-1)
                for k in range(len(xs)):
                    cv2.circle(canvas, (int(xs[k]), int(ys[k])), draw_point_radius_px, COL_CYAN, -1, lineType=cv2.LINE_AA)

                # Label
                cv2.putText(canvas, f"Cell {cid}  {axis}-rot {ang:03d}°",
                            (8, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, TXT_COL, 1, cv2.LINE_AA)

                frames.append(canvas)

            out_path = os.path.join(out_dir, f"cell_{cid:03d}_spin{axis}.mp4")
            try:
                imageio.mimsave(out_path,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps, format='FFMPEG')
                print(f"Saved: {out_path}")
            except TypeError:
                out_gif = out_path.replace(".mp4", ".gif")
                imageio.mimsave(out_gif,
                                [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                                fps=fps)
                print(f"Saved (GIF): {out_gif}")

        # Make the three spins
        make_spin("Z")
        make_spin("Y")
        make_spin("X")
        exported += 1

    print(f"Exported {exported} per-cell rotation videos (X, Y, Z) → {out_dir}")

save_cell_spin_videos_allaxes(
    image_2,            # raw Ch2 volume (Z,Y,X)
    cell_seg,
    blobs, df,
    out_dir="cell_spin_videos",
    angles=tuple(range(0, 360, 10)),
    pad=8, fps=12
)
"""
# ==========================================
# 9) 3D screenshots (XY, YZ, XZ)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original stack (Ch1) as MP4
# ==========================================
try:
    raw_stack = np.array(image, dtype=np.float32)
    raw_norm = (255 * (raw_stack - raw_stack.min()) / (raw_stack.ptp() + 1e-8)).astype(np.uint8)

    frames_raw = []
    for z in range(raw_norm.shape[0]):
        frame_gray = raw_norm[z]
        frame_bgr = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2BGR)
        frames_raw.append(frame_bgr)

    mp4_name = "original_raw_ch1.mp4"
    imageio.mimsave(mp4_name, frames_raw, fps=8, format='FFMPEG')
    print(f"Saved: {mp4_name}")
except Exception as e:
    print("MP4 export of original stack failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def to_uint8_grayscale(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return (np.zeros_like(vol, dtype=np.uint8))
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    raw_stack_u8 = to_uint8_grayscale(image)      # left panel (raw Ch1)
    seg_base_u8  = to_uint8_grayscale(image_2)    # right base (Ch2)

    Z, H, W = raw_stack_u8.shape
    fps = 8

    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=fps, format="FFMPEG")

    for z in range(Z):
        left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
        base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)

        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr


        # --- draw ONLY in-cell lysosomes on the right panel ---
        if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                # draw only if this blob center is inside the cell on this slice
                if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                    cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)  # yellow (in-cell only)
        # NOTE: no drawing at all for outside points


        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(raw_stack_u8.shape[0]):
            left_bgr = cv2.cvtColor(raw_stack_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(seg_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            # --- draw ONLY in-cell lysosomes on the right panel ---
            if 'blobs' in globals() and len(blobs) > 0 and 'cell_mask' in globals():
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    # draw only if this blob center is inside the cell on this slice
                    if 0 <= y < H and 0 <= x < W and cell_mask[z, y, x]:
                        cv2.circle(right_bgr, (x, y), r, (255, 255, 0), 2)  # yellow (in-cell only)
            # NOTE: no drawing at all for outside points

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 12) Run viewer
# ==========================================
napari.run()